In [ ]:
HTML(read(open("style.css"), String))

# Testing Avoid Repetion when Engine have a better position

In [ ]:
using Pkg
# Pkg.add("NBInclude")
using NBInclude
# Pkg.add("Chess")
using Chess

In [ ]:
@nbinclude("PGN_Export.ipynb")

In [ ]:
@nbinclude("Utility.ipynb")

In [ ]:
# Function from Play.ipynb (We decided not to import because of the interactive function that is implemented from Play.ipynb)
function playMove(game::Game, aBoard::AdvBoard, cache::Dict{UInt64, Tuple{String, Int64, Int64}})
    starttime = time()
    #---------------------- Setup Engine ----------------------------
    # RANDOM ENGINE
    # eval, move = 0, generateRandomMove(game)
    
    # MINIMAX ENGINE
    # eval, move = minimax(board(game), currentcurrent_boardscorescore, 4, cache)

    # ALPHA BETA PRUNING ENGINE
    # eval, move = alphaBetaPruning(board(game), current_boardscore, 4, cache)

    # ITERATIVE DEEPENING ENGINE
    println("Cache Length before calculation: ", length(cache))
    eval, move = iterativeDeepening(aBoard, 8, cache, true, 0.0, true)

    #----------------------------------------------------------------
    
    println("The Engine needs to be calculated the bestmove: $(time() - starttime) seconds")
    println("Current board score: $(aBoard.score)")
    println("Evaluation of engine: $eval")
    println("Current entries in Cache: $(length(cache))")
    println("Computer play: $move")
    clearCache(aBoard, move, cache)
    domove!(game, move)
    domoveAdv!(aBoard, move)
    cleanCacheBecauseRep!(aBoard, cache)
    return aBoard
end

Function for simulating a game where white tries to draw by stalemate or repetition.

In [ ]:
# Define a set about the moves (a8,b8,b7)
# 1. Loop above game is over (checkmate or draw)
# 2. Make Schnittmenge from moves to possible moves
# 3. Move First Move from Schnittmenge
# 3.1 If no Schnittmenge, give User input
# 4. Calculate with Iterative Deepening the best move for black and move it.
# 5. Repeat 1-4 until game is over
function testAvoidRepForWhite(game)
    preferredMoves = Set([movefromstring("b8a8"), movefromstring("a8b8"), movefromstring("a8b7"), movefromstring("b7a8"), movefromstring("b7b8"),movefromstring("b8b7") ])
    aBoard = AdvBoard(deepcopy(board(game)))
    cache = initCache()
    while !isterminal(game)
        printGame(game)
        possibleMoves = Set(moves(board(game)))
        intersection = intersect(preferredMoves, possibleMoves)
        if length(intersection) > 0
            move = first(intersection)
        else
            move = first(possibleMoves)
        end
        clearCache(aBoard, move, cache)
        # cache = initCache()
        #println("Cache: ", aBoard.repCounter)
        #println("Board: ", aBoard.state)
        #println("zobrist hash: ", zobrist_hash(aBoard.state))
        #println("amount: ", aBoard.repCounter[zobrist_hash(aBoard.state)])
        domove!(game, move)
        domoveAdv!(aBoard, move)
        cleanCacheBecauseRep!(aBoard, cache)
        print(aBoard.repCounter)
        try
            playMove(game, aBoard, cache)
        catch e 
            bt = backtrace()
            msg = sprint(showerror, e, bt)
            println(msg)
            return aBoard, cache
        end
    end
    println("--------------------------------------")
    println("Cache: ", aBoard.repCounter)
    println("Board: ", aBoard.state)
    println("zobrist hash: ", zobrist_hash(aBoard.state))
    println("amount: ", aBoard.repCounter[zobrist_hash(aBoard.state)])
    setWin(game, "")
    println(game)
    saveGameToPGN(game)
    return aBoard, cache
end

In this test below, we check that black should be able to checkmate white. Black should not stalemate or make a draw by repetition.

In [ ]:
b1 = fromfen("K7/P6p/2q5/3p1k2/8/8/8/8 w - - 0 1")
myA, myCache = testAvoidRepForWhite(Game(b1)) # We excpect Black to checkmate white